In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd


/home/shashank/miniconda3/envs/tag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

/home/shashank/miniconda3/envs/tag/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
data = pd.read_csv("data/papers.csv")
# data = data[data["Abstract"].notna()]

# add a ai_generated_tag column
data["ai_generated_tag"] = ""

In [9]:
all_tags_dict = {
"Deep reinforcement learning is a machine learning approach that combines deep neural networks with reinforcement learning (RL) algorithms to enable agents to learn optimal actions through trial and error in complex environments.":"Deep reinforcement learning",

"Informative path planning is a robotics and AI technique that aims to find optimal paths for agents or vehicles by considering the most informative locations or viewpoints along the way to gather valuable data or observations.":"Informative path planning",

"Simulation is the imitation or emulation of the operation of a real-world process or system over time, allowing for analysis, testing, and understanding of its behavior without directly interacting with the actual system.":"Simulation",

"Learning dynamics a methodology or technique where simulation parameters are estimated ": "Learning for simulation",

"Localization in robotics refers to the process of determining a robot's precise position and orientation within its environment, typically using sensors and mapping techniques to navigate and perform tasks accurately.":"Localization",

"Mapping in robotics refers to the process of creating a spatial representation of an environment using sensors and data, enabling a robot to understand and navigate its surroundings.":"Mapping",

"simultaneous localization and mapping is a computational technique used in robotics to enable a robot to build a map of its environment while simultaneously determining its own position within that map in real-time.":"SLAM",

"Multi robot simultaneous localization and mapping in robotics is the collaborative process where multiple robots work together to create a map of their environment while simultaneously determining their own positions within that map.":"Multi-robot SLAM",

"Multi robot coordination and planning in robotics refers to the process of enabling multiple robots to work together efficiently and collaboratively to achieve common goals by optimizing their actions and movements in a coordinated manner.":"Multi-robot coordination and planning",

"Deformable manipulation in robotics refers to the capability of a robot to interact with and manipulate objects that have flexible or deformable properties, such as textiles or soft materials, by adapting its grasp and manipulation strategies to accommodate their changing shapes.":"Deformable manipulation",

"Learning from demonstrations in robotics is a process where a robot acquires new skills or behaviors by observing and imitating human or expert demonstrations.":"Learning from demonstrations",

"Motion planning in robotics is the process of determining a sequence of feasible movements for a robot to navigate from its current position to a desired goal while avoiding obstacles and adhering to constraints.":"Motion planning",

"Meta learning in robotics and machine learning is a technique where a model learns how to learn, enabling it to adapt and generalize to new tasks or environments more efficiently by leveraging prior learning experiences.":"Meta learning",
}

all_tags = list(all_tags_dict.keys())
tags_embeddings = sentence_transformer_model.encode(all_tags, convert_to_tensor=True)

In [11]:
threshold = 0.2

for i, row in data.iterrows():
    abstract = row["Abstract"]
    if pd.isna(abstract):
        continue

    # abstract_sentences = abstract.split(".")
    abstract_sentences = [abstract]

    # Compute embedding for both abstract and tags
    abstract_embedding = sentence_transformer_model.encode(abstract_sentences, convert_to_tensor=True)

    cosine_scores = util.dot_score(tags_embeddings, abstract_embedding)

    cosine_scores = cosine_scores.max(1).values
    # cosine_scores = cosine_scores.mean(1)

    # for j in range(len(tags_embeddings)):
    #     print("\n{}   Score: {:.4f}".format(all_tags[j], cosine_scores[j][0]))

    # Sort the scores for tags
    sorted_indices = cosine_scores.argsort(descending=True)
    # get the tags with scores above the threshold
    sorted_indices = sorted_indices[cosine_scores[sorted_indices] > threshold]#[0:1]

    top_5_tags_dict = {all_tags_dict[all_tags[idx]]: str(cosine_scores[idx].item())[:4] for idx in sorted_indices}

    title = row["Title"]
    print("\nTitle: {}: tags: {}".format(title, top_5_tags_dict))

    if len(top_5_tags_dict) == 0:
        print("No tags found for this paper")
        continue

    # add the tags to the dataframe
    list_of_tags = list(top_5_tags_dict.keys())
    data.at[i, "ai_generated_tag"] = ", ".join(list_of_tags)


Title: GranularGym: High Performance Simulation for Robotic Tasks with Granular  Materials.: tags: {'Learning for simulation': '0.35', 'Deep reinforcement learning': '0.29', 'Deformable manipulation': '0.28', 'Motion planning': '0.27', 'Meta learning': '0.27', 'Learning from demonstrations': '0.27', 'Simulation': '0.25', 'Mapping': '0.23', 'Informative path planning': '0.23', 'Multi-robot SLAM': '0.21', 'SLAM': '0.21', 'Multi-robot coordination and planning': '0.21'}

Title: IndustReal: Transferring Contact-Rich Assembly Tasks from Simulation  to Reality.: tags: {'Deep reinforcement learning': '0.51', 'Learning from demonstrations': '0.48', 'Meta learning': '0.43', 'Motion planning': '0.42', 'Deformable manipulation': '0.37', 'Multi-robot coordination and planning': '0.35', 'Informative path planning': '0.34', 'Learning for simulation': '0.34', 'Localization': '0.31', 'Mapping': '0.30', 'SLAM': '0.27', 'Multi-robot SLAM': '0.26', 'Simulation': '0.25'}

Title: Fast and Scalable Signal 

In [15]:
data[["Title", "ai_generated_tag"]]

,Title,ai_generated_tag
0,GranularGym: High Performance Simulation for R...,Motion planning
1,IndustReal: Transferring Contact-Rich Assembly...,"Simulation, Learning from demonstrations, Deep..."
2,Fast and Scalable Signal Inference for Active ...,"Motion planning, Multi-robot coordination and ..."
3,Language-Informed Transfer Learning for Embodi...,"Learning for simulation, Deep reinforcement le..."
4,RREx-BoT: Remote Referring Expressions with a ...,"SLAM, Multi-robot SLAM, Learning from demonstr..."
...,...,...
462,The Design and Control of a Prototype Quadrupe...,"Motion planning, Informative path planning"
463,Mobility evaluation of a wheeled microrover us...,"Simulation, Motion planning, Multi-robot coord..."
464,Mission Reachability for Extraterrestrial Rovers.,
465,On the Development of EMG Control for a Prosth...,"Simulation, Deformable manipulation"


In [16]:
# save the dataframe
data.to_csv("data/papers_with_ai_tags.csv", index=False)